In [1]:
%matplotlib inline


UMAP and ISOMap on the MNIST dataset
--------------------------------

the result will be saved in umap_isomap_measure.csv

The MNIST database of handwritten digits with 784 features, raw data available at: http://yann.lecun.com/exdb/mnist/. It can be split in a training set of the first 60,000 examples, and a test set of 10,000 examples

It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. the images were centered in a 28x28 image by computing the center of mass of the pixels, and translating the image so as to position this point at the center of the 28x28 field.
https://www.openml.org/d/554

In this code I run with 10,000 sample

In [2]:
from sklearn.datasets import fetch_openml
# from sklearn.datasets import load_digits
mnist = fetch_openml('mnist_784', version=1, cache=True)
# mnist = load_digits()
import sys 
import trace 
import threading
from threading import Thread

In [3]:
import psutil
import os
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import multiprocessing as mp
import collections
import sys
import subprocess
import pandas as pd

# Get the process ID of 
# the current process 
pid = os.getpid() 
memFrame = {'rss': [], 'vms': [],'num_page_faults': [],'peak_wset': [],'wset': [],'peak_paged_pool': [],'paged_pool': [],'peak_nonpaged_pool': [],'nonpaged_pool': [],'pagefile': [],'peak_pagefile': [],'private': [],'cpu_times':[],'cpu_percent':[]}
memf = pd.DataFrame(data=memFrame)
  
# Print the process ID of 
# the current process 
print(pid)  
p = psutil.Process(pid)

21624


In [4]:
def measure():
    info = []
#     memoryUse = p.memory_info()[0]/2.**30
#     print('memory use:', memoryUse)
    p.name()  # execute internal routine once collecting multiple info
    mem = p.memory_info()
    for m in mem:
        info.append(m)
    info.extend([p.cpu_times(),p.cpu_percent()])
    df_length = len(memf)
    memf.loc[df_length] = info

In [5]:
import umap
from sklearn.manifold import Isomap
import matplotlib.pyplot as plt
import seaborn as sns
import psutil
from datetime import datetime
import pandas as pd
import time
import os

sns.set(context="paper", style="white")



def umap_f():
    reducer = umap.UMAP(n_components=2)
    embedding = reducer.fit_transform(mnist.data[1:10000])
    print('umap: done')
    measure()
def isomap_f():
    embedding = Isomap(n_components=2)
    X_transformed = embedding.fit_transform(mnist.data[1:10000])
    print('isomap: done')
    measure()

In [6]:
measure()

In [7]:
t1=Thread( target=umap_f)
t2=Thread( target=isomap_f)

t1.start()
t2.start()

t1.join()
t2.join()

umap: done
isomap: done


In [8]:
memf

,rss,vms,num_page_faults,peak_wset,wset,peak_paged_pool,paged_pool,peak_nonpaged_pool,nonpaged_pool,pagefile,peak_pagefile,private,cpu_times,cpu_percent
0,6.348308e+08,6.050652e+08,429028.0,1.123172e+09,6.348308e+08,796920.0,796920.0,1191600.0,171104.0,6.050652e+08,1.119146e+09,6.050652e+08,"(27.609375, 1.640625, 0.0, 0.0)",0.0
1,9.014559e+08,9.085010e+08,538802.0,1.123172e+09,9.014559e+08,802256.0,802256.0,1235384.0,184288.0,9.085010e+08,1.119146e+09,9.085010e+08,"(153.9375, 2.15625, 0.0, 0.0)",197.7
2,2.338468e+09,2.323280e+09,1323256.0,3.942289e+09,2.338468e+09,802256.0,802256.0,1235384.0,181840.0,2.323280e+09,3.930354e+09,2.323280e+09,"(276.78125, 3.53125, 0.0, 0.0)",103.5


In [9]:
memf.to_csv('umap_isomap_measure.csv',index=False)